# Enigma machine
Specification from [Codes and Ciphers](http://www.codesandciphers.org.uk/enigma/rotorspec.htm) page.

Example Enigma machines from [Louise Dale](http://enigma.louisedade.co.uk/enigma.html) (full simulation) and [EnigmaCo](http://enigmaco.de/enigma/enigma.html) (good animation of the wheels, but no ring settings).

There's also the nice Enigma simulator for Android by [Franklin Heath](https://franklinheath.co.uk/2012/02/04/our-first-app-published-enigma-simulator/), available on the [Google Play store](https://play.google.com/store/apps/details?id=uk.co.franklinheath.enigmasim&hl=en_GB).

In [1]:
import string
import collections

cat = ''.join

def clean(text): return cat(l.lower() for l in text if l in string.ascii_letters)

def pos(letter): 
    if letter in string.ascii_lowercase:
        return ord(letter) - ord('a')
    elif letter in string.ascii_uppercase:
        return ord(letter) - ord('A')
    else:
        return ''
    
def unpos(number): return chr(number % 26 + ord('a'))

In [2]:
wheel_i_spec = 'ekmflgdqvzntowyhxuspaibrcj'
wheel_ii_spec = 'ajdksiruxblhwtmcqgznpyfvoe'
wheel_iii_spec = 'bdfhjlcprtxvznyeiwgakmusqo'
wheel_iv_spec = 'esovpzjayquirhxlnftgkdcmwb'
wheel_v_spec = 'vzbrgityupsdnhlxawmjqofeck'
wheel_vi_spec = 'jpgvoumfyqbenhzrdkasxlictw'
wheel_vii_spec = 'nzjhgrcxmyswboufaivlpekqdt'
wheel_viii_spec = 'fkqhtlxocbjspdzramewniuygv'
beta_wheel_spec = 'leyjvcnixwpbqmdrtakzgfuhos'
gamma_wheel_spec = 'fsokanuerhmbtiycwlqpzxvgjd'

wheel_i_pegs = ['q']
wheel_ii_pegs = ['e']
wheel_iii_pegs = ['v']
wheel_iv_pegs = ['j']
wheel_v_pegs = ['z']
wheel_vi_pegs = ['z', 'm']
wheel_vii_pegs = ['z', 'm']
wheel_viii_pegs = ['z', 'm']

reflector_b_spec = 'ay br cu dh eq fs gl ip jx kn mo tz vw'
reflector_c_spec = 'af bv cp dj ei go hy kr lz mx nw tq su'

In [3]:
class LetterTransformer(object):
    def __init__(self, specification, raw_transform=False):
        if raw_transform:
            transform = specification
        else:
            transform = self.parse_specification(specification)
        self.validate_transform(transform)
        self.make_transform_map(transform)
    
    def parse_specification(self, specification):
        return list(zip(string.ascii_lowercase, clean(specification)))
        # return specification
    
    def validate_transform(self, transform):
        """A set of pairs, of from-to"""
        if len(transform) != 26:
            raise ValueError("Transform specification has {} pairs, requires 26".
                format(len(transform)))
        for p in transform:
            if len(p) != 2:
                raise ValueError("Not all mappings in transform "
                    "have two elements")
        if len(set([p[0] for p in transform])) != 26:
            raise ValueError("Transform specification must list 26 origin letters") 
        if len(set([p[1] for p in transform])) != 26:
            raise ValueError("Transform specification must list 26 destination letters") 

    def make_empty_transform(self):
        self.forward_map = [0] * 26
        self.backward_map = [0] * 26
            
    def make_transform_map(self, transform):
        self.make_empty_transform()
        for p in transform:
            self.forward_map[pos(p[0])] = pos(p[1])
            self.backward_map[pos(p[1])] = pos(p[0])
        return self.forward_map, self.backward_map
    
    def forward(self, letter):
        if letter in string.ascii_lowercase:
            return unpos(self.forward_map[pos(letter)])
        else:
            return ''
                
    def backward(self, letter):
        if letter in string.ascii_lowercase:
            return unpos(self.backward_map[pos(letter)])
        else:
            return ''

In [4]:
tmap = [('z', 'a')] + [(l, string.ascii_lowercase[i+1]) for i, l in enumerate(string.ascii_lowercase[:-1])]
tmap

[('z', 'a'),
 ('a', 'b'),
 ('b', 'c'),
 ('c', 'd'),
 ('d', 'e'),
 ('e', 'f'),
 ('f', 'g'),
 ('g', 'h'),
 ('h', 'i'),
 ('i', 'j'),
 ('j', 'k'),
 ('k', 'l'),
 ('l', 'm'),
 ('m', 'n'),
 ('n', 'o'),
 ('o', 'p'),
 ('p', 'q'),
 ('q', 'r'),
 ('r', 's'),
 ('s', 't'),
 ('t', 'u'),
 ('u', 'v'),
 ('v', 'w'),
 ('w', 'x'),
 ('x', 'y'),
 ('y', 'z')]

In [5]:
cat(collections.OrderedDict.fromkeys('zyxwc' + string.ascii_lowercase))

'zyxwcabdefghijklmnopqrstuv'

In [6]:
tmap2 = list(zip(string.ascii_lowercase, cat(collections.OrderedDict.fromkeys('zyxwc' + string.ascii_lowercase))))

In [7]:
lt = LetterTransformer(tmap, raw_transform = True)
assert(lt.forward_map == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 0])
assert(lt.backward_map == [25, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
lt.forward_map, lt.backward_map

([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  0],
 [25,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24])

In [8]:
lt = LetterTransformer(cat(collections.OrderedDict.fromkeys('zyxwc' + string.ascii_lowercase)))
assert(lt.forward_map == [25, 24, 23, 22, 2, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])
assert(lt.backward_map == [5, 6, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 3, 2, 1, 0])
assert(cat(lt.forward(l) for l in string.ascii_lowercase) == 'zyxwcabdefghijklmnopqrstuv')
assert(cat(lt.backward(l) for l in string.ascii_lowercase) == 'fgehijklmnopqrstuvwxyzdcba')
lt.forward_map, lt.backward_map

([25,
  24,
  23,
  22,
  2,
  0,
  1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21],
 [5,
  6,
  4,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  3,
  2,
  1,
  0])

In [9]:
cat(lt.forward(l) for l in string.ascii_lowercase)

'zyxwcabdefghijklmnopqrstuv'

In [10]:
cat(lt.backward(l) for l in string.ascii_lowercase)

'fgehijklmnopqrstuvwxyzdcba'

In [11]:
class Plugboard(LetterTransformer):
    def parse_specification(self, specification):
        return [tuple(clean(p)) for p in specification.split()]
    
    def validate_transform(self, transform):
        """A set of pairs, of from-to"""
        for p in transform:
            if len(p) != 2:
                raise ValueError("Not all mappings in transform"
                    "have two elements")
    
    def make_empty_transform(self):
        self.forward_map = list(range(26))
        self.backward_map = list(range(26))
        
    def make_transform_map(self, transform):
        expanded_transform = transform + [tuple(reversed(p)) for p in transform]
        return super(Plugboard, self).make_transform_map(expanded_transform)

In [12]:
pb = Plugboard([('a', 'z'), ('b', 'y')], raw_transform=True)

In [13]:
cat(pb.forward(l) for l in string.ascii_lowercase)

'zycdefghijklmnopqrstuvwxba'

In [14]:
cat(pb.backward(l) for l in string.ascii_lowercase)

'zycdefghijklmnopqrstuvwxba'

In [15]:
pb = Plugboard('az by')

In [16]:
cat(pb.forward(l) for l in string.ascii_lowercase), cat(pb.backward(l) for l in string.ascii_lowercase)

('zycdefghijklmnopqrstuvwxba', 'zycdefghijklmnopqrstuvwxba')

In [17]:
pb = Plugboard('ua pf rq so ni ey bg hl tx zj'.upper())
assert(pb.forward_map == pb.backward_map)
assert(pb.forward_map == [20, 6, 2, 3, 24, 15, 1, 11, 13, 25, 10, 7, 12, 8, 18, 5, 17, 16, 14, 23, 0, 21, 22, 19, 4, 9])
assert(cat(pb.forward(l) for l in string.ascii_lowercase) == 'ugcdypblnzkhmisfrqoxavwtej')
assert(cat(pb.backward(l) for l in string.ascii_lowercase) == 'ugcdypblnzkhmisfrqoxavwtej')
cat(pb.forward(l) for l in string.ascii_lowercase), cat(pb.backward(l) for l in string.ascii_lowercase)

('ugcdypblnzkhmisfrqoxavwtej', 'ugcdypblnzkhmisfrqoxavwtej')

In [18]:
class Reflector(Plugboard):
    def validate_transform(self, transform):
        if len(transform) != 13:
            raise ValueError("Reflector specification has {} pairs, requires 13".
                format(len(transform)))
        if len(set([p[0] for p in transform] + 
                    [p[1] for p in transform])) != 26:
            raise ValueError("Reflector specification does not contain 26 letters")
        try:
            super(Reflector, self).validate_transform(transform)
        except ValueError as v:
            raise ValueError("Not all mappings in reflector have two elements")

In [19]:
# reflector_b_text = '(AY) (BR) (CU) (DH) (EQ) (FS) (GL) (IP) (JX) (KN) (MO) (TZ) (VW)'
reflector_b_l = [tuple(clean(p)) for p in reflector_b_spec.split()]
reflector_b_l

[('a', 'y'),
 ('b', 'r'),
 ('c', 'u'),
 ('d', 'h'),
 ('e', 'q'),
 ('f', 's'),
 ('g', 'l'),
 ('i', 'p'),
 ('j', 'x'),
 ('k', 'n'),
 ('m', 'o'),
 ('t', 'z'),
 ('v', 'w')]

In [20]:
reflector_b = Reflector(reflector_b_spec)
assert(reflector_b.forward_map == reflector_b.backward_map)
assert(reflector_b.forward_map == [24, 17, 20, 7, 16, 18, 11, 3, 15, 23, 13, 6, 14, 10, 12, 8, 4, 1, 5, 25, 2, 22, 21, 9, 0, 19])
assert(cat(reflector_b.forward(l) for l in string.ascii_lowercase) == 'yruhqsldpxngokmiebfzcwvjat')
assert(cat(reflector_b.backward(l) for l in string.ascii_lowercase) == 'yruhqsldpxngokmiebfzcwvjat')

In [21]:
cat(reflector_b.forward(l) for l in string.ascii_lowercase)

'yruhqsldpxngokmiebfzcwvjat'

In [22]:
reflector_c = Reflector(reflector_c_spec)

In [23]:
cat(reflector_c.forward(l) for l in string.ascii_lowercase)

'fvpjiaoyedrzxwgctkuqsbnmhl'

In [24]:
class SimpleWheel(LetterTransformer):
    def __init__(self, transform, position='a', raw_transform=False):
        super(SimpleWheel, self).__init__(transform, raw_transform)
        self.set_position(position)
        
    def __getattribute__(self,name):
        if name=='position_l':
            return unpos(self.position)
        else:
            return object.__getattribute__(self, name)
    
    def set_position(self, position):
        self.position = ord(position) - ord('a')
    
    def forward(self, letter):
        if letter in string.ascii_lowercase:
            return unpos((self.forward_map[(pos(letter) + self.position) % 26] - self.position))
        else:
            return ''
                
    def backward(self, letter):
        if letter in string.ascii_lowercase:
            return unpos((self.backward_map[(pos(letter) + self.position) % 26] - self.position))
        else:
            return ''
        
    def advance(self):
        self.position = (self.position + 1) % 26
        return self.position

In [25]:
rotor_1_transform = list(zip(string.ascii_lowercase, 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'.lower()))
rotor_1_transform

[('a', 'e'),
 ('b', 'k'),
 ('c', 'm'),
 ('d', 'f'),
 ('e', 'l'),
 ('f', 'g'),
 ('g', 'd'),
 ('h', 'q'),
 ('i', 'v'),
 ('j', 'z'),
 ('k', 'n'),
 ('l', 't'),
 ('m', 'o'),
 ('n', 'w'),
 ('o', 'y'),
 ('p', 'h'),
 ('q', 'x'),
 ('r', 'u'),
 ('s', 's'),
 ('t', 'p'),
 ('u', 'a'),
 ('v', 'i'),
 ('w', 'b'),
 ('x', 'r'),
 ('y', 'c'),
 ('z', 'j')]

In [26]:
rotor_1_transform = list(zip(string.ascii_lowercase, 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'.lower()))
wheel_1 = SimpleWheel(rotor_1_transform, raw_transform=True)
assert(cat(wheel_1.forward(l) for l in string.ascii_lowercase) == 'ekmflgdqvzntowyhxuspaibrcj')
assert(cat(wheel_1.backward(l) for l in string.ascii_lowercase) == 'uwygadfpvzbeckmthxslrinqoj')

In [27]:
cat(wheel_1.forward(l) for l in string.ascii_lowercase), cat(wheel_1.backward(l) for l in string.ascii_lowercase)

('ekmflgdqvzntowyhxuspaibrcj', 'uwygadfpvzbeckmthxslrinqoj')

In [28]:
wheel_1.position_l

'a'

In [29]:
wheel_2 = SimpleWheel(wheel_ii_spec)
assert(cat(wheel_2.forward(l) for l in string.ascii_lowercase) == 'ajdksiruxblhwtmcqgznpyfvoe')
assert(cat(wheel_2.backward(l) for l in string.ascii_lowercase) == 'ajpczwrlfbdkotyuqgenhxmivs')

In [30]:
cat(wheel_2.forward(l) for l in string.ascii_lowercase), cat(wheel_2.backward(l) for l in string.ascii_lowercase)

('ajdksiruxblhwtmcqgznpyfvoe', 'ajpczwrlfbdkotyuqgenhxmivs')

In [31]:
wheel_3 = SimpleWheel(wheel_iii_spec)
wheel_3.set_position('a')
wheel_3.advance()
assert(cat(wheel_3.forward(l) for l in string.ascii_lowercase) == 'cegikboqswuymxdhvfzjltrpna')
assert(cat(wheel_3.backward(l) for l in string.ascii_lowercase) == 'zfaobrcpdteumygxhwivkqjnls')
assert(wheel_3.position == 1)
assert(wheel_3.position_l == 'b')

for _ in range(24): wheel_3.advance()
assert(wheel_3.position == 25)
assert(wheel_3.position_l == 'z')
assert(cat(wheel_3.forward(l) for l in string.ascii_lowercase) == 'pcegikmdqsuywaozfjxhblnvtr')
assert(cat(wheel_3.backward(l) for l in string.ascii_lowercase) == 'nubhcqdterfvgwoaizjykxmslp')

wheel_3.advance()
assert(wheel_3.position == 0)
assert(wheel_3.position_l == 'a')
assert(cat(wheel_3.forward(l) for l in string.ascii_lowercase) == 'bdfhjlcprtxvznyeiwgakmusqo')
assert(cat(wheel_3.backward(l) for l in string.ascii_lowercase) == 'tagbpcsdqeufvnzhyixjwlrkom')

cat(wheel_3.forward(l) for l in string.ascii_lowercase), cat(wheel_3.backward(l) for l in string.ascii_lowercase)

('bdfhjlcprtxvznyeiwgakmusqo', 'tagbpcsdqeufvnzhyixjwlrkom')

In [32]:
class Wheel(SimpleWheel):
    def __init__(self, transform, ring_peg_letters, ring_setting=1, position='a', raw_transform=False):
        self.ring_peg_letters = ring_peg_letters
        self.ring_setting = ring_setting
        super(Wheel, self).__init__(transform, position=position, raw_transform=raw_transform)
        self.set_position(position)
        
    def __getattribute__(self,name):
        if name=='position_l':
            return unpos(self.position + self.ring_setting - 1)
        else:
            return object.__getattribute__(self, name)

    def set_position(self, position):
        self.position = (pos(position) - self.ring_setting + 1) % 26
        # self.position_l = position
        self.peg_positions = [(pos(p) - pos(position)) % 26  for p in self.ring_peg_letters]
        
    def advance(self):
        super(Wheel, self).advance()
        self.peg_positions = [(p - 1) % 26 for p in self.peg_positions]
        # self.position_l = unpos(self.position + self.ring_setting - 1)
        return self.position

In [33]:
wheel_3 = Wheel(wheel_iii_spec, wheel_iii_pegs, position='b', ring_setting=1)

In [34]:
wheel_3.position, wheel_3.peg_positions

(1, [20])

In [35]:
wheel_6 = Wheel(wheel_vi_spec, wheel_vi_pegs, position='b', ring_setting=3)
wheel_6.position, wheel_6.peg_positions

(25, [24, 11])

In [36]:
for _ in range(27):
    wheel_6.advance()
    print(wheel_6.position, wheel_6.peg_positions)

0 [23, 10]
1 [22, 9]
2 [21, 8]
3 [20, 7]
4 [19, 6]
5 [18, 5]
6 [17, 4]
7 [16, 3]
8 [15, 2]
9 [14, 1]
10 [13, 0]
11 [12, 25]
12 [11, 24]
13 [10, 23]
14 [9, 22]
15 [8, 21]
16 [7, 20]
17 [6, 19]
18 [5, 18]
19 [4, 17]
20 [3, 16]
21 [2, 15]
22 [1, 14]
23 [0, 13]
24 [25, 12]
25 [24, 11]
0 [23, 10]


In [37]:
wheel_3 = Wheel(wheel_iii_spec, wheel_iii_pegs, position='b', ring_setting=1)
assert(wheel_3.position == 1)
assert(wheel_3.peg_positions == [20])
assert(wheel_3.position_l == 'b')
wheel_3.advance()
assert(wheel_3.position == 2)
assert(wheel_3.peg_positions == [19])
assert(wheel_3.position_l == 'c')

In [38]:
wheel_6 = Wheel(wheel_vi_spec, wheel_vi_pegs, position='b', ring_setting=3)
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'xkqhwpvngzrcfoiaselbtymjdu')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'ptlyrmidoxbswhnfckquzgeavj')
assert(wheel_6.position == 25)
assert(11 in wheel_6.peg_positions)
assert(24 in wheel_6.peg_positions)
assert(wheel_6.position_l == 'b')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'jpgvoumfyqbenhzrdkasxlictw')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'skxqlhcnwarvgmebjptyfdzuio')
assert(wheel_6.position == 0)
assert(10 in wheel_6.peg_positions)
assert(23 in wheel_6.peg_positions)
assert(wheel_6.position_l == 'c')

for _ in range(22): wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'mgxantkzsyqjcufirldvhoewbp')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'dymswobuplgraevzkqifntxcjh')
assert(wheel_6.position == 22)
assert(1 in wheel_6.peg_positions)
assert(14 in wheel_6.peg_positions)
assert(wheel_6.position_l == 'y')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'fwzmsjyrxpibtehqkcugndvaol')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'xlrvnatokfqzduyjphemswbigc')
assert(wheel_6.position == 23)
assert(0 in wheel_6.peg_positions)
assert(13 in wheel_6.peg_positions)
assert(wheel_6.position_l == 'z')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'vylrixqwohasdgpjbtfmcuznke')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'kqumzsnjepyctxiogdlrvahfbw')
assert(wheel_6.position == 24)
assert(25 in wheel_6.peg_positions)
assert(12 in wheel_6.peg_positions)
assert(wheel_6.position_l == 'a')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'xkqhwpvngzrcfoiaselbtymjdu')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'ptlyrmidoxbswhnfckquzgeavj')
assert(wheel_6.position == 25)
assert(24 in wheel_6.peg_positions)
assert(11 in wheel_6.peg_positions)
assert(wheel_6.position_l == 'b')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'jpgvoumfyqbenhzrdkasxlictw')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'skxqlhcnwarvgmebjptyfdzuio')
assert(wheel_6.position == 0)
assert(23 in wheel_6.peg_positions)
assert(10 in wheel_6.peg_positions)
assert(wheel_6.position_l == 'c')

In [39]:
wheel_6.position, wheel_6.position_l, wheel_6.peg_positions

(0, 'c', [23, 10])

In [40]:
class Enigma(object):
    def __init__(self, reflector_spec,
                 left_wheel_spec, left_wheel_pegs,
                 middle_wheel_spec, middle_wheel_pegs,
                 right_wheel_spec, right_wheel_pegs,
                 left_ring_setting, middle_ring_setting, right_ring_setting,
                 plugboard_setting):
        self.reflector = Reflector(reflector_spec)
        self.left_wheel = Wheel(left_wheel_spec, left_wheel_pegs, ring_setting=left_ring_setting)
        self.middle_wheel = Wheel(middle_wheel_spec, middle_wheel_pegs, ring_setting=middle_ring_setting)
        self.right_wheel = Wheel(right_wheel_spec, right_wheel_pegs, ring_setting=right_ring_setting)
        self.plugboard = Plugboard(plugboard_setting)
        
    def __getattribute__(self,name):
        if name=='wheel_positions':
            return self.left_wheel.position, self.middle_wheel.position, self.right_wheel.position 
        elif name=='wheel_positions_l':
            return self.left_wheel.position_l, self.middle_wheel.position_l, self.right_wheel.position_l 
        elif name=='peg_positions':
            return self.left_wheel.peg_positions, self.middle_wheel.peg_positions, self.right_wheel.peg_positions
        else:
            return object.__getattribute__(self, name)

    
    def set_wheels(self, left_wheel_position, middle_wheel_position, right_wheel_position):
        self.left_wheel.set_position(left_wheel_position)
        self.middle_wheel.set_position(middle_wheel_position)
        self.right_wheel.set_position(right_wheel_position)
        
    def lookup(self, letter):
        a = self.plugboard.forward(letter)
        b = self.right_wheel.forward(a)
        c = self.middle_wheel.forward(b)
        d = self.left_wheel.forward(c)
        e = self.reflector.forward(d)
        f = self.left_wheel.backward(e)
        g = self.middle_wheel.backward(f)
        h = self.right_wheel.backward(g)
        i = self.plugboard.backward(h)
        return i
    
    def advance(self):
        advance_middle = False
        advance_left = False
        if 0 in self.right_wheel.peg_positions:
            advance_middle = True
        if 0 in self.middle_wheel.peg_positions:
            advance_left = True
            advance_middle = True
        self.right_wheel.advance()
        if advance_middle: self.middle_wheel.advance()
        if advance_left: self.left_wheel.advance()
            
    def encipher_letter(self, letter):
        self.advance()
        return self.lookup(letter)
    
    def encipher(self, message):
        enciphered = ''
        for letter in clean(message):
            enciphered += self.encipher_letter(letter)
        return enciphered

In [41]:
enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_pegs,
                wheel_ii_spec, wheel_ii_pegs,
                wheel_iii_spec, wheel_iii_pegs,
                1, 1, 1,
                '')

In [42]:
enigma.lookup('a')

'u'

In [43]:
enigma.lookup('u')

'a'

In [44]:
cat(enigma.lookup(l) for l in string.ascii_lowercase)

'uejobtpzwcnsrkdgvmlfaqiyxh'

In [45]:
enigma.set_wheels('a', 'a', 'a')
for i in range(26):
    print(i, '::', 
          enigma.left_wheel.position_l, enigma.middle_wheel.position_l, enigma.right_wheel.position_l, ';',
          enigma.left_wheel.peg_positions, enigma.middle_wheel.peg_positions, enigma.right_wheel.peg_positions, 
         cat(enigma.lookup(l) for l in string.ascii_lowercase))
    enigma.advance()

0 :: a a a ; [16] [4] [21] uejobtpzwcnsrkdgvmlfaqiyxh
1 :: a a b ; [16] [4] [20] baqmfexihswpdytlcvjozrkgnu
2 :: a a c ; [16] [4] [19] djralkwpobfeyqihncxzvugsmt
3 :: a a d ; [16] [4] [18] zlejcuitgdmbkonsvxphfqyrwa
4 :: a a e ; [16] [4] [17] gcblwtakqzhdosmxiunfryepvj
5 :: a a f ; [16] [4] [16] osnirgfmdpvuhcajwebxlkqtzy
6 :: a a g ; [16] [4] [15] wymvnqzjlhoicekuftxrpdasbg
7 :: a a h ; [16] [4] [14] cjafkdztpbeuormiwnvhlsqyxg
8 :: a a i ; [16] [4] [13] xijuyslvbczgnmqwotfrdhpaek
9 :: a a j ; [16] [4] [12] lfzrwbytjisaovmuxdkhpneqgc
10 :: a a k ; [16] [4] [11] tkezcqynuwbpvhslfxoaimjrgd
11 :: a a l ; [16] [4] [10] kiwfnduxbsaotelqpvjmgrchzy
12 :: a a m ; [16] [4] [9] sfkutbpoxycrnmhgwlaedzqijv
13 :: a a n ; [16] [4] [8] baqwlkhgrsfextpocijnvudmzy
14 :: a a o ; [16] [4] [7] teofbdzxqkjyrscvimnawpuhlg
15 :: a a p ; [16] [4] [6] mhypswrbzxqvaondkgeutlfjci
16 :: a a q ; [16] [4] [5] cpasnrhgkuixzevbyfdwjotlqm
17 :: a a r ; [16] [4] [4] dlfatcjwygvbnmzrxpueskhqio
18 :: a a s ; [16] [4] [3]

In [46]:
enigma.set_wheels('a', 'a', 't')
assert(enigma.wheel_positions == (0, 0, 19))
assert(cat(enigma.wheel_positions_l) == 'aat')
assert(enigma.peg_positions == ([16], [4], [2]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'puvioztjdhxmlyeawsrgbcqknf')

enigma.advance()
assert(enigma.wheel_positions == (0, 0, 20))
assert(cat(enigma.wheel_positions_l) == 'aau')
assert(enigma.peg_positions == ([16], [4], [1]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'baigpldqcowfyzjehvtsxrkumn')

enigma.advance()
assert(enigma.wheel_positions == (0, 0, 21))
assert(cat(enigma.wheel_positions_l) == 'aav')
assert(enigma.peg_positions == ([16], [4], [0]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'mnvfydiwgzsoablrxpkutchqej')

enigma.advance()
assert(enigma.wheel_positions == (0, 1, 22))
assert(cat(enigma.wheel_positions_l) == 'abw')
assert(enigma.peg_positions == ([16], [3], [25]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ulfopcykswhbzvderqixanjtgm')

enigma.advance()
assert(enigma.wheel_positions == (0, 1, 23))
assert(cat(enigma.wheel_positions_l) == 'abx')
assert(enigma.peg_positions == ([16], [3], [24]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'qmwftdyovursbzhxaklejicpgn')

enigma.advance()
assert(enigma.wheel_positions == (0, 1, 24))
assert(cat(enigma.wheel_positions_l) == 'aby')
assert(enigma.peg_positions == ([16], [3], [23]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'oljmzxrvucybdqasngpwihtfke')


In [47]:
enigma.set_wheels('a', 'd', 't')
assert(enigma.wheel_positions == (0, 3, 19))
assert(cat(enigma.wheel_positions_l) == 'adt')
assert(enigma.peg_positions == ([16], [1], [2]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'zcbpqxwsjiuonmldethrkygfva')

enigma.advance()
assert(enigma.wheel_positions == (0, 3, 20))
assert(cat(enigma.wheel_positions_l) == 'adu')
assert(enigma.peg_positions == ([16], [1], [1]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ehprawjbngotxikcsdqlzyfmvu')

enigma.advance()
assert(enigma.wheel_positions == (0, 3, 21))
assert(cat(enigma.wheel_positions_l) == 'adv')
assert(enigma.peg_positions == ([16], [1], [0]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'eqzxarpihmnvjkwgbfuyslodtc')

enigma.advance()
assert(enigma.wheel_positions == (0, 4, 22))
assert(cat(enigma.wheel_positions_l) == 'aew')
assert(enigma.peg_positions == ([16], [0], [25]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'qedcbtpluzmhkongavwfirsyxj')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 23))
assert(cat(enigma.wheel_positions_l) == 'bfx')
assert(enigma.peg_positions == ([15], [25], [24]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'iwuedhsfazqxytvrkpgncoblmj')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 24))
assert(cat(enigma.wheel_positions_l) == 'bfy')
assert(enigma.peg_positions == ([15], [25], [23]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'baknstqzrmcxjdvygiefwoulph')

print(enigma.wheel_positions, enigma.wheel_positions_l, enigma.peg_positions, 
         cat(enigma.lookup(l) for l in string.ascii_lowercase))


(1, 5, 24) ('b', 'f', 'y') ([15], [25], [23]) baknstqzrmcxjdvygiefwoulph


In [48]:
enigma.set_wheels('a', 'a', 'a')
ct = enigma.encipher('testmessage')
assert(ct == 'olpfhnvflyn')
ct

'olpfhnvflyn'

In [49]:
enigma.set_wheels('a', 'd', 't')
ct = enigma.encipher('testmessage')
assert(ct == 'lawnjgpwjik')
ct

'lawnjgpwjik'

In [50]:

for i in range(26):
    enigma.advance()
    print('enigma.advance()')
    print("assert(enigma.wheel_positions == {})".format(enigma.wheel_positions))
    print("assert(cat(enigma.wheel_positions_l) == '{}')".format(cat(enigma.wheel_positions_l)))
    print("assert(enigma.peg_positions == {})".format(enigma.peg_positions))
    print("assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(enigma.lookup(l) for l in string.ascii_lowercase)))
    print()

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 5))
assert(cat(enigma.wheel_positions_l) == 'bff')
assert(enigma.peg_positions == ([15], [25], [16]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'zpqiogfsdlmjkyebcvhxwrutna')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 6))
assert(cat(enigma.wheel_positions_l) == 'bfg')
assert(enigma.peg_positions == ([15], [25], [15]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'fjmnwayslbxicdpouthrqzekgv')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 7))
assert(cat(enigma.wheel_positions_l) == 'bfh')
assert(enigma.peg_positions == ([15], [25], [14]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'csafzdyloxuhnmitwvbpkrqjge')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 8))
assert(cat(enigma.wheel_positions_l) == 'bfi')
assert(enigma.peg_positions == ([15], [25], [13]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'kihyvulcbtagwr

In [51]:
enigma.set_wheels('a', 'd', 't')
ct = enigma.encipher('hellothere')
assert(ct == 'bahxvfrpdc')
ct

'bahxvfrpdc'

In [52]:
enigma.set_wheels('b', 'd', 'q')
ct = enigma.encipher('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
assert(ct == 'kvmmwrlqlqsqpeugjrcxzwpfyiyybwloewrouvkpoztceuwtfjzqwpbqldttsr')
assert(enigma.left_wheel.position_l == 'c')
assert(enigma.middle_wheel.position_l == 'h')
assert(enigma.right_wheel.position_l == 'a')
ct

'kvmmwrlqlqsqpeugjrcxzwpfyiyybwloewrouvkpoztceuwtfjzqwpbqldttsr'

In [53]:
enigma.left_wheel.position_l

'c'

In [54]:
# Setting sheet line 31 from http://www.codesandciphers.org.uk/enigma/enigma3.htm
# Enigma simulation settings are 
# http://enigma.louisedade.co.uk/enigma.html?m3;b;b153;AFTX;AJFE;AU-BG-EY-FP-HL-IN-JZ-OS-QR-TX
w_enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_pegs,
                wheel_v_spec, wheel_v_pegs,
                wheel_iii_spec, wheel_iii_pegs,
                6, 20, 24,
                'ua pf rq so ni ey bg hl tx zj')

In [55]:
# Setting sheet line 31 from http://www.codesandciphers.org.uk/enigma/enigma3.htm
# Enigma simulation settings are 
# http://enigma.louisedade.co.uk/enigma.html?m3;b;b153;AFTX;AJFE;AU-BG-EY-FP-HL-IN-JZ-OS-QR-TX
enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_pegs,
                wheel_v_spec, wheel_v_pegs,
                wheel_iii_spec, wheel_iii_pegs,
                6, 20, 24,
                'ua pf rq so ni ey bg hl tx zj')

In [56]:
enigma.set_wheels('j', 'e', 'u')

enigma.advance()
assert(enigma.wheel_positions == (4, 11, 24))
assert(cat(enigma.wheel_positions_l) == 'jev')
assert(enigma.peg_positions == ([7], [21], [0]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'mvqjlyowkdieasgzcunxrbhtfp')

enigma.advance()
assert(enigma.wheel_positions == (4, 12, 25))
assert(cat(enigma.wheel_positions_l) == 'jfw')
assert(enigma.peg_positions == ([7], [20], [25]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'sjolzuyvrbwdpxcmtiaqfhknge')

enigma.advance()
assert(enigma.wheel_positions == (4, 12, 0))
assert(cat(enigma.wheel_positions_l) == 'jfx')
assert(enigma.peg_positions == ([7], [20], [24]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'qrxedkoywufmlvgsabpzjnicht')

enigma.advance()
assert(enigma.wheel_positions == (4, 12, 1))
assert(cat(enigma.wheel_positions_l) == 'jfy')
assert(enigma.peg_positions == ([7], [20], [23]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'hpsukliagqefwvtbjxcodnmrzy')

enigma.advance()
assert(enigma.wheel_positions == (4, 12, 2))
assert(cat(enigma.wheel_positions_l) == 'jfz')
assert(enigma.peg_positions == ([7], [20], [22]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'zevnbpyqowrtxdifhkulscjmga')


In [57]:
enigma.set_wheels('i', 'd', 'z')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 3))
assert(cat(enigma.wheel_positions_l) == 'ida')
assert(enigma.peg_positions == ([8], [22], [21]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ikhpqrvcambzjondefwyxgsutl')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 4))
assert(cat(enigma.wheel_positions_l) == 'idb')
assert(enigma.peg_positions == ([8], [22], [20]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'cdabskhgzwfmlqvunyexpojtri')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 5))
assert(cat(enigma.wheel_positions_l) == 'idc')
assert(enigma.peg_positions == ([8], [22], [19]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'pcbwiqhgemyvjsuaftnroldzkx')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 6))
assert(cat(enigma.wheel_positions_l) == 'idd')
assert(enigma.peg_positions == ([8], [22], [18]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'xcbfvdnouptmlghjzwykierasq')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 7))
assert(cat(enigma.wheel_positions_l) == 'ide')
assert(enigma.peg_positions == ([8], [22], [17]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'xfvglbdynuseriwqpmkzjcoaht')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 8))
assert(cat(enigma.wheel_positions_l) == 'idf')
assert(enigma.peg_positions == ([8], [22], [16]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'tfpqlbouynsewjgcdxkahzmriv')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 9))
assert(cat(enigma.wheel_positions_l) == 'idg')
assert(enigma.peg_positions == ([8], [22], [15]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'cjaunvlwtbygzexrspqidfhokm')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 10))
assert(cat(enigma.wheel_positions_l) == 'idh')
assert(enigma.peg_positions == ([8], [22], [14]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'yltxkrqvowebzpingfucshjdam')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 11))
assert(cat(enigma.wheel_positions_l) == 'idi')
assert(enigma.peg_positions == ([8], [22], [13]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'myktluzrnxceaiqsohpdfwvjbg')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 12))
assert(cat(enigma.wheel_positions_l) == 'idj')
assert(enigma.peg_positions == ([8], [22], [12]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'pynjrmiugdqxfcvakewzhoslbt')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 13))
assert(cat(enigma.wheel_positions_l) == 'idk')
assert(enigma.peg_positions == ([8], [22], [11]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'mwvedyplnoxhaijgrqtszcbkfu')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 14))
assert(cat(enigma.wheel_positions_l) == 'idl')
assert(enigma.peg_positions == ([8], [22], [10]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'qcbrfeutvoxpnmjladzhgiykws')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 15))
assert(cat(enigma.wheel_positions_l) == 'idm')
assert(enigma.peg_positions == ([8], [22], [9]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'dnoahryetsmukbcvwfjilpqzgx')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 16))
assert(cat(enigma.wheel_positions_l) == 'idn')
assert(enigma.peg_positions == ([8], [22], [8]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'nidcfehgbqsovalyjzkxwmutpr')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 17))
assert(cat(enigma.wheel_positions_l) == 'ido')
assert(enigma.peg_positions == ([8], [22], [7]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'joifxdulcarhzpbntkwqgysevm')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 18))
assert(cat(enigma.wheel_positions_l) == 'idp')
assert(enigma.peg_positions == ([8], [22], [6]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ptnlsxvozmwdjchayuebrgkfqi')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 19))
assert(cat(enigma.wheel_positions_l) == 'idq')
assert(enigma.peg_positions == ([8], [22], [5]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'slwopzqnmxybihdeguavrtcjkf')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 20))
assert(cat(enigma.wheel_positions_l) == 'idr')
assert(enigma.peg_positions == ([8], [22], [4]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'hcbedwlamzogixkytsrqvufnpj')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 21))
assert(cat(enigma.wheel_positions_l) == 'ids')
assert(enigma.peg_positions == ([8], [22], [3]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'odxbjwzrmelkisavuhnyqpfctg')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 22))
assert(cat(enigma.wheel_positions_l) == 'idt')
assert(enigma.peg_positions == ([8], [22], [2]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'udgbfeclrwnhxksvtioqapjmzy')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 23))
assert(cat(enigma.wheel_positions_l) == 'idu')
assert(enigma.peg_positions == ([8], [22], [1]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'nrdczqxmowvshaiufblypkjgte')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 24))
assert(cat(enigma.wheel_positions_l) == 'idv')
assert(enigma.peg_positions == ([8], [22], [0]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'hkifjdoacebqtzgulyvmpsxwrn')

enigma.advance()
assert(enigma.wheel_positions == (3, 11, 25))
assert(cat(enigma.wheel_positions_l) == 'iew')
assert(enigma.peg_positions == ([8], [21], [25]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'yptzuhofqvnmlkgbixwcejsrad')

enigma.advance()
assert(enigma.wheel_positions == (3, 11, 0))
assert(cat(enigma.wheel_positions_l) == 'iex')
assert(enigma.peg_positions == ([8], [21], [24]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'vkdcwhqfjibzsptngumoraeyxl')

enigma.advance()
assert(enigma.wheel_positions == (3, 11, 1))
assert(cat(enigma.wheel_positions_l) == 'iey')
assert(enigma.peg_positions == ([8], [21], [23]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'wenpbqrouxlkychdfgzvitajms')


In [58]:
enigma.set_wheels('i', 'd', 'z')
ct = enigma.encipher('verylongtestmessagewithanextrabitofmessageforgoodmeasure')
assert(ct == 'gstsegeqdrthkfwesljjomfvcqwcfspxpfqqmewvddybarzwubxtpejz')
assert(enigma.wheel_positions == (3, 12, 6))
assert(cat(enigma.wheel_positions_l) == 'ifd')
assert(enigma.peg_positions == ([8], [20], [18]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'urygzpdmxtwshqvfnbljaokice')

enigma.set_wheels('i', 'd', 'z')
pt = enigma.encipher('gstsegeqdrthkfwesljjomfvcqwcfspxpfqqmewvddybarzwubxtpejz')
assert(pt == 'verylongtestmessagewithanextrabitofmessageforgoodmeasure')

pt, enigma.wheel_positions, enigma.wheel_positions_l, enigma.peg_positions, cat(enigma.lookup(l) for l in string.ascii_lowercase)

('verylongtestmessagewithanextrabitofmessageforgoodmeasure',
 (3, 12, 6),
 ('i', 'f', 'd'),
 ([8], [20], [18]),
 'urygzpdmxtwshqvfnbljaokice')

In [59]:
enigma.set_wheels('i', 'd', 'z')

for i in range(26):
    enigma.advance()
    print('enigma.advance()')
    print("assert(enigma.wheel_positions == {})".format(enigma.wheel_positions))
    print("assert(cat(enigma.wheel_positions_l) == '{}')".format(cat(enigma.wheel_positions_l)))
    print("assert(enigma.peg_positions == {})".format(enigma.peg_positions))
    print("assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(enigma.lookup(l) for l in string.ascii_lowercase)))
    print()

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 3))
assert(cat(enigma.wheel_positions_l) == 'ida')
assert(enigma.peg_positions == ([8], [22], [21]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ikhpqrvcambzjondefwyxgsutl')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 4))
assert(cat(enigma.wheel_positions_l) == 'idb')
assert(enigma.peg_positions == ([8], [22], [20]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'cdabskhgzwfmlqvunyexpojtri')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 5))
assert(cat(enigma.wheel_positions_l) == 'idc')
assert(enigma.peg_positions == ([8], [22], [19]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'pcbwiqhgemyvjsuaftnroldzkx')

enigma.advance()
assert(enigma.wheel_positions == (3, 10, 6))
assert(cat(enigma.wheel_positions_l) == 'idd')
assert(enigma.peg_positions == ([8], [22], [18]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'xcbfvdnouptmlg